In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import iqr
from statsmodels.formula.api import ols
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# Create bar plot
plt.style.use('seaborn-v0_8-poster')
#plt.style.use('seaborn-v0_8')
fontsize = 11
plt.rcParams.update({
    "font.size": fontsize,          # Global font size
    "axes.labelsize": fontsize,     # X and Y axis labels
    "axes.titlesize": fontsize,     # Title (if used)
    "xtick.labelsize": fontsize,    # X-axis tick labels
    "ytick.labelsize": fontsize,    # Y-axis tick labels
    "legend.fontsize": fontsize,    # Legend font size
})

In [2]:
project_dir = '/Users/xiaoqianxiao/projects/NARSAD'
SCR_dir = os.path.join(project_dir, 'EDR')
fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')

In [3]:
def rename_roi(roi):
    cleaned_roi = roi.replace('_mask_resampled', '').replace('_mask', '').replace('_resampled', '')
    sides = ['left', 'right']
    regions = ['hippocampus', 'amygdala', 'vmpfc', 'insula', 'ACC']  # Add more as needed
    
    side = None
    region = None
    for s in sides:
        if s in cleaned_roi:
            side = s
            break
    for r in regions:
        if r in cleaned_roi:
            region = r
            break
    if side and region:
        return f"{side}_{region}"
    else:
        return cleaned_roi

In [4]:
# Define directories
root_dir = '/Users/xiaoqianxiao/projects'
project_name = 'NARSAD'

sub_no_MRI_phase2 = ['N102', 'N208']
sub_no_MRI_phase3 = ['N102', 'N208', 'N120']

SCR_dir = os.path.join(root_dir, project_name, 'EDR')
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')

# Load behavioral data
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')

group_info_df_phase2 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase2)]
group_info_df_phase3 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase3)]

fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
df_fMRI['roi_name'] = df_fMRI['ROI'].apply(rename_roi)
# Define the desired order for 'roi_name'
roi_order = ['ACC', 
             'left_vmpfc','right_vmpfc',
             'left_insula','right_insula', 
             'left_hippocampus', 'right_hippocampus', 
             'right_amygdala', 'left_amygdala'
]

df_fMRI['roi_name'] = pd.Categorical(df_fMRI['roi_name'], categories=roi_order, ordered=True)

df_fMRI_phase2 = df_fMRI.loc[df_fMRI['Task']=='phase2'].copy()
df_fMRI_phase3 = df_fMRI.loc[df_fMRI['Task']=='phase3'].copy()

In [5]:
df_fMRI_phase2['subID'] = df_fMRI_phase2['Subject'].apply(lambda x: group_info_df_phase2.iloc[x - 1]['subID'])
df_fMRI_phase3['subID'] = df_fMRI_phase3['Subject'].apply(lambda x: group_info_df_phase3.iloc[x - 1]['subID'])
df_phase2 = df_fMRI_phase2.merge(group_info_df_phase2, how='left', left_on='subID', right_on='subID')
df_phase2 = df_phase2[df_phase2['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)
df_phase3 = df_fMRI_phase3.merge(group_info_df_phase3, how='left', left_on='subID', right_on='subID')
df_phase3 = df_phase3[df_phase3['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)

In [6]:
df_current = df_phase3

In [7]:
# reinstatement
for roi in df_current['roi_name'].unique():
    # t-test: ?> 0
    print(f'T-test results for {roi}: Greater CCR than CCS?')
    results = []
    for (group, drug, gender), sub_df in df.groupby(['group', 'Drug', 'Gender'], observed=True):
        t_stat, p_value = stats.ttest_1samp(sub_df['z_fMRI'], 0, nan_policy='omit')  # Ignore NaNs
        
        results.append({
            'Group': group,
            'Drug': drug,
            'Gender': gender,
            't_stat': t_stat,
            'p_value': p_value,
            'n': len(sub_df)  # Sample size
        })
    df_results = pd.DataFrame(results)
    df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
    print(df_results)

T-test results for right_hippocampus: Greater CCR than CCS?


NameError: name 'df' is not defined

In [8]:
# reinstatement
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('reinstatement results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin([4,6]))].copy()
    df_tmp.loc[df_tmp['Contrast']==4, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==6, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

reinstatement results for right_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.9021
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.527
Time:                        15:01:53   Log-Likelihood:                -139.00
No. Observations:                 101   AIC:                             298.0
Df Residuals:                      91   BIC:                             324.1
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

,Condition,Effect,p_value
86,left_insula,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.058965
6,right_hippocampus,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.062422
44,right_amygdala,C(group)[T.Patients]:C(Drug)[T.Placebo],0.066402
46,right_amygdala,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.097066
16,right_insula,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.108936
...,...,...,...
76,left_amygdala,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.933047
63,ACC,C(Gender)[T.Male],0.940462
75,left_amygdala,C(group)[T.Patients]:C(Gender)[T.Male],0.950606
67,ACC,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.953064


for the entire dataset. LINS, RHIP, RAMY Drug * gender; rAMY, drug * group

In [22]:
df_drug

,Effect,ROI,b,SE,t,p,CI_lower,CI_upper
6,C(Drug)[T.Placebo],ACC,-0.225413,0.405346,-0.556101,0.579507,-1.030584,0.579757
7,C(Drug)[T.Placebo],left_amygdala,0.229443,0.406530,0.564395,0.573874,-0.578078,1.036965
3,C(Drug)[T.Placebo],left_hippocampus,0.338320,0.404328,0.836745,0.404928,-0.464829,1.141469
8,C(Drug)[T.Placebo],left_insula,0.565606,0.402598,1.404890,0.163458,-0.234105,1.365318
5,C(Drug)[T.Placebo],left_vmpfc,0.473789,0.403571,1.173991,0.243462,-0.327855,1.275434
4,C(Drug)[T.Placebo],right_amygdala,0.498702,0.391866,1.272635,0.206390,-0.279691,1.277096
0,C(Drug)[T.Placebo],right_hippocampus,0.354089,0.398631,0.888263,0.376741,-0.437743,1.145921
1,C(Drug)[T.Placebo],right_insula,0.200325,0.402034,0.498279,0.619490,-0.598267,0.998917
2,C(Drug)[T.Placebo],right_vmpfc,-0.260801,0.396671,-0.657475,0.512535,-1.048740,0.527137


In [9]:
# reinstatement in only placebo
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('reinstatement results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin([4,6]))].copy()
    df_tmp.loc[df_tmp['Contrast']==4, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==6, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI     
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'ROI': roi,
            'Effect': effect_idx,
            'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
        })
    
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p', ascending=True)

reinstatement results for right_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.112
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     1.111
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.368
Time:                        15:02:07   Log-Likelihood:                -67.975
No. Observations:                  50   AIC:                             147.9
Df Residuals:                      44   BIC:                             159.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

,ROI,Effect,b,SE,t,p,CI_lower,CI_upper
25,right_amygdala,C(group)[T.Patients],-1.017177,0.410601,-2.477287,0.017151,-1.844689,-0.189665
27,right_amygdala,C(group)[T.Patients]:C(Gender)[T.Male],1.328114,0.582312,2.280759,0.027461,0.154541,2.501688
3,right_hippocampus,C(group)[T.Patients]:C(Gender)[T.Male],1.196031,0.596278,2.005828,0.051047,-0.005688,2.397750
26,right_amygdala,C(Gender)[T.Male],-0.772857,0.386052,-2.001948,0.051479,-1.550894,0.005180
2,right_hippocampus,C(Gender)[T.Male],-0.773229,0.395311,-1.956003,0.056832,-1.569926,0.023467
8,right_insula,C(Gender)[T.Male],-0.722646,0.392519,-1.841047,0.072363,-1.513716,0.068424
50,left_insula,C(Gender)[T.Male],-0.664957,0.398101,-1.670322,0.101956,-1.467277,0.137363
33,left_vmpfc,C(group)[T.Patients]:C(Gender)[T.Male],0.899548,0.613039,1.467359,0.149390,-0.335950,2.135047
47,left_amygdala,demo_age,-0.031688,0.022114,-1.432971,0.158934,-0.076256,0.012879
1,right_hippocampus,C(group)[T.Patients],-0.585739,0.420449,-1.393129,0.170579,-1.433098,0.261619


In [20]:
# placebo data, gender * group effect
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin([4,6]))].copy()
    df_tmp.loc[df_tmp['Contrast']==4, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==6, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                    col='group', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in reinstatement', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'reinstatement_placedo_data_interaction_effect_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

right_hippocampus
right_insula
right_vmpfc
left_hippocampus
right_amygdala
left_vmpfc
ACC
left_amygdala
left_insula


In [10]:
# placebo data, gender effect
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin([4,6]))].copy()
    df_tmp.loc[df_tmp['Contrast']==4, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==6, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                    #col='group', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in reinstatement', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'reinstatement_placedo_data_gender_effect_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

right_hippocampus
right_insula
right_vmpfc
left_hippocampus
right_amygdala
left_vmpfc
ACC
left_amygdala
left_insula


In [11]:
# placebo data, group effect
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin([4,6]))].copy()
    df_tmp.loc[df_tmp['Contrast']==4, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==6, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='group', y='z_fMRI', hue='CS_Type',
                    #col='group', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in reinstatement', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'reinstatement_placebo_data_group_effect_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

right_hippocampus
right_insula
right_vmpfc
left_hippocampus
right_amygdala
left_vmpfc
ACC
left_amygdala
left_insula


In [12]:
for roi in df_current['roi_name'].unique():
    # bar plots for reinstatement
    #roi = 'ACC_mask_resampled'
    print(f'{roi}')
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin([4,6]))].copy()
    df_tmp.loc[df_tmp['Contrast']==4, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==6, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    g = sns.catplot(x='drug', 
                    y='z_fMRI', hue='CS_Type', 
                    #col='group', 
                    #row='drug', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    g.legend.set_title(f'{roi}')
    g.figure.set_dpi(300)
    #plt.show()
    plt.close()

right_hippocampus
right_insula
right_vmpfc
left_hippocampus
right_amygdala
left_vmpfc
ACC
left_amygdala
left_insula


In [13]:
# reinstatement in each group
results_list = []
drug_results_list = []
for group in df['group'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'reinstatement results for {roi} {group}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                          (df_current['group'] == group) & 
                          (df_current['Contrast'].isin([4,6]))].copy()
        
        df_tmp.loc[df_tmp['Contrast']==4, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==6, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                        columns='CS_Type', 
                        values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
        #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        
        combined_col = f'{roi}_{group}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Effect': effect_idx,
                'p_value': p_value
            })
        conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
        for effect_idx in result_ols.params.index:
            if effect_idx == 'C(Drug)[T.Placebo]': 
                drug_results_list.append({
                    'Effect': effect_idx,
                    'group': group,
                    'ROI': roi,
                    'b': result_ols.params[effect_idx],  # Coefficient
                    'SE': result_ols.bse[effect_idx],    # Standard error
                    't': result_ols.tvalues[effect_idx], # t-statistic
                    'p': result_ols.pvalues[effect_idx],  # p-value
                    'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                    'CI_upper': conf_int.loc[effect_idx, 1]
                })
    
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

reinstatement results for right_hippocampus Patients
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.8447
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.526
Time:                        15:02:51   Log-Likelihood:                -67.232
No. Observations:                  49   AIC:                             146.5
Df Residuals:                      43   BIC:                             157.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

,Condition,Effect,p_value
107,left_insula_Controls,demo_age,0.015456
106,left_insula_Controls,Avoidance_centered,0.039711
81,right_amygdala_Controls,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.042400
71,right_vmpfc_Controls,demo_age,0.045029
105,left_insula_Controls,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.055723
...,...,...,...
38,ACC_Patients,C(Gender)[T.Male],0.970994
62,right_insula_Controls,C(Gender)[T.Male],0.972426
3,right_hippocampus_Patients,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.980585
51,left_insula_Patients,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.990045


in group specific dataset, drug effect was significant in Controls in ACC

In [14]:
df_drug

,Effect,group,ROI,b,SE,t,p,CI_lower,CI_upper
15,C(Drug)[T.Placebo],Controls,ACC,-0.229000,0.404436,-0.566221,0.573996,-1.043087,0.585087
6,C(Drug)[T.Placebo],Patients,ACC,0.081266,0.413131,0.196706,0.844984,-0.751893,0.914424
16,C(Drug)[T.Placebo],Controls,left_amygdala,0.211576,0.404923,0.522509,0.603823,-0.603491,1.026643
7,C(Drug)[T.Placebo],Patients,left_amygdala,-0.192067,0.399869,-0.480324,0.633432,-0.998480,0.614347
12,C(Drug)[T.Placebo],Controls,left_hippocampus,0.277600,0.406267,0.683294,0.497849,-0.540173,1.095372
3,C(Drug)[T.Placebo],Patients,left_hippocampus,0.499976,0.403101,1.240324,0.221581,-0.312955,1.312906
8,C(Drug)[T.Placebo],Patients,left_insula,0.112593,0.410041,0.274591,0.784946,-0.714333,0.939520
17,C(Drug)[T.Placebo],Controls,left_insula,0.570813,0.362724,1.573683,0.122414,-0.159313,1.300938
14,C(Drug)[T.Placebo],Controls,left_vmpfc,0.478901,0.404379,1.184290,0.242382,-0.335070,1.292873
5,C(Drug)[T.Placebo],Patients,left_vmpfc,-0.058022,0.399560,-0.145214,0.885221,-0.863812,0.747768


In [15]:
import pandas as pd
from scipy.stats import zscore, ttest_ind

desired_comparisons = [
    'Female_Oxytocin vs. Female_Placebo',
    'Male_Oxytocin vs. Male_Placebo'
]

t_test_results_list = []

for group in df['group'].unique():
    for roi in df_current['roi_name'].unique():
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                           (df_current['group'] == group) & 
                           (df_current['Contrast'].isin([4, 6]))].copy()
        
        df_tmp.loc[df_tmp['Contrast'] == 4, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast'] == 6, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID', 'Drug', 'Gender', 'group', 'Avoidance', 'demo_age'], 
                          columns='CS_Type', values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])  # Offset of 1 assumed intentional
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
        
        group_labels = df['Gender'].astype(str) + "_" + df['Drug'].astype(str)
        
        for comparison in desired_comparisons:
            g1, g2 = comparison.split(' vs. ')
            if g1 in group_labels.values and g2 in group_labels.values:  # Check if both groups exist
                group1_data = df[group_labels == g1]['z_fMRI']
                group2_data = df[group_labels == g2]['z_fMRI']
                
                t_stat, p_value = ttest_ind(group1_data, group2_data, nan_policy='omit')
                
                mean_diff = group1_data.mean() - group2_data.mean()
                se = ((group1_data.var() / group1_data.count()) + (group2_data.var() / group2_data.count())) ** 0.5
                
                t_test_results_list.append({
                    'ROI': roi,
                    'Group': group,
                    'Comparison': comparison,
                    'Mean_Diff': mean_diff,
                    'SE': se,
                    't': t_stat,
                    'p': p_value
                })

# Create DataFrame with all results
df_t_test = pd.DataFrame(t_test_results_list)
df_t_test = df_t_test.sort_values('p', ascending=True)  # Sort by p-value

In [16]:
df_t_test

,ROI,Group,Comparison,Mean_Diff,SE,t,p
5,right_vmpfc,Controls,Male_Oxytocin vs. Male_Placebo,0.880722,0.362874,2.483609,0.020384
3,right_insula,Controls,Male_Oxytocin vs. Male_Placebo,0.834310,0.421157,2.001629,0.056753
1,right_hippocampus,Controls,Male_Oxytocin vs. Male_Placebo,0.690842,0.359367,1.934754,0.064892
8,right_amygdala,Controls,Female_Oxytocin vs. Female_Placebo,-0.540165,0.313598,-1.737085,0.095192
17,left_insula,Controls,Male_Oxytocin vs. Male_Placebo,0.621819,0.391210,1.631141,0.115915
16,left_insula,Controls,Female_Oxytocin vs. Female_Placebo,-0.628426,0.366236,-1.625425,0.117133
10,left_vmpfc,Controls,Female_Oxytocin vs. Female_Placebo,-0.516816,0.373543,-1.334364,0.194608
13,ACC,Controls,Male_Oxytocin vs. Male_Placebo,0.506488,0.419029,1.174752,0.251618
9,right_amygdala,Controls,Male_Oxytocin vs. Male_Placebo,0.441507,0.431289,0.995894,0.329237
6,left_hippocampus,Controls,Female_Oxytocin vs. Female_Placebo,-0.322488,0.401105,-0.824261,0.417908


In [17]:
results_list = []
# reinstatement, in each gender
for g in df['Gender'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'reinstatement results for {roi} {g}')
        # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
        df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Gender'] == g) & (df_current['Contrast'].isin([2,3]))].copy()
        df_tmp.loc[df_tmp['Contrast']==2, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==3, 'CS_Type'] = 'CSR'
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(group) * C(Drug) + Avoidance_centered + demo_age", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        combined_col = f'{roi}_{g}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Effect': effect_idx,
                'p_value': p_value
            })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

reinstatement results for right_hippocampus Male
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.200
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     2.005
Date:                Wed, 12 Mar 2025   Prob (F-statistic):             0.0988
Time:                        15:03:07   Log-Likelihood:                -60.128
No. Observations:                  46   AIC:                             132.3
Df Residuals:                      40   BIC:                             143.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------

,Condition,Effect,p_value
2,right_hippocampus_Male,C(Drug)[T.Placebo],0.030221
14,right_vmpfc_Male,C(Drug)[T.Placebo],0.038198
8,right_insula_Male,C(Drug)[T.Placebo],0.062546
77,left_hippocampus_Female,demo_age,0.062656
81,right_amygdala_Female,C(group)[T.Patients]:C(Drug)[T.Placebo],0.102522
...,...,...,...
58,right_hippocampus_Female,Avoidance_centered,0.933993
89,left_vmpfc_Female,demo_age,0.965143
7,right_insula_Male,C(group)[T.Patients],0.966767
76,left_hippocampus_Female,Avoidance_centered,0.983988


in Gender specific data subset, drug effect in male in RHIP, RVMPFC, RIGHT INS

In [18]:
import pandas as pd 
from statsmodels.formula.api import ols
from scipy.stats import zscore

results_list = []
for g in df['Gender'].unique():
    for group in df_current['group'].unique():
        for roi in df_current['roi_name'].unique():
            print(f'reinstatement results for {roi} {g}')
            df_tmp = df_current[(df_current['roi_name'] == roi) & 
                              (df_current['Gender'] == g) & 
                              (df_current['group'] == group) & 
                              (df_current['Contrast'].isin([4,6]))].copy()
            
            df_tmp.loc[df_tmp['Contrast']==4, 'CS_Type'] = 'CSS'
            df_tmp.loc[df_tmp['Contrast']==6, 'CS_Type'] = 'CSR'
            
            df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                            columns='CS_Type', 
                            values='Mean_COPE_Value').reset_index()
            
            df['fMRI'] = df['CSR'] - df['CSS']
            df['z_fMRI'] = 1 + zscore(df['fMRI'])
            df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
            df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
            
            df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
            
            model_ols = ols("z_fMRI ~ C(Drug) + Avoidance_centered + demo_age", data=df)
            
            #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
            result_ols = model_ols.fit()
            print(result_ols.summary())
            
            combined_col = f'{roi}_{group}_{g}'
            
            for effect_idx in result_ols.pvalues.index:
                p_value = result_ols.pvalues[effect_idx]
                results_list.append({
                    'Condition': combined_col,
                    'Effect': effect_idx,
                    'p_value': p_value
                })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

reinstatement results for right_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.102
Method:                 Least Squares   F-statistic:                    0.1374
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.937
Time:                        15:03:13   Log-Likelihood:                -40.912
No. Observations:                  29   AIC:                             89.82
Df Residuals:                      25   BIC:                             95.29
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

,Condition,Effect,p_value
43,right_insula_Controls_Female,demo_age,0.007460
71,left_insula_Controls_Female,demo_age,0.009079
51,left_hippocampus_Controls_Female,demo_age,0.017120
59,left_vmpfc_Controls_Female,demo_age,0.037277
70,left_insula_Controls_Female,Avoidance_centered,0.051356
23,left_vmpfc_Patients_Female,demo_age,0.063088
47,right_vmpfc_Controls_Female,demo_age,0.076471
42,right_insula_Controls_Female,Avoidance_centered,0.090447
53,right_amygdala_Controls_Female,C(Drug)[T.Placebo],0.100698
11,right_vmpfc_Patients_Female,demo_age,0.118418


In [21]:
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin([4, 6]))].copy()
    df_tmp.loc[df_tmp['Contrast'] == 4, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast'] == 6, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='Drug', y='z_fMRI', hue='CS_Type',
                    order=['Placebo', 'Oxytocin'], 
                    col='group', 
                    row='Gender', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in reinstatement', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'reinstatement_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

right_hippocampus
right_insula
right_vmpfc
left_hippocampus
right_amygdala
left_vmpfc
ACC
left_amygdala
left_insula
